<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/v.2/w_okada's_Voice_Changer_version_2_x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### w-okada's Voice Changer version 2.x | **Google Colab**

## READ ME - VERY IMPORTANT
This is an attempt to run [Realtime Voice Changer](https://github.com/w-okada/voice-changer) on Google Colab, still not perfect but is totally usable, you can use the following settings for better results:

If you're using a index: `f0: RMVPE_ONNX | Chunk: 24000 or higher | Extra: 7680`\
If you're not using a index: `f0: RMVPE_ONNX | Chunk: 24000 or higher | Extra: 7680`\
**Don't forget to select your Colab GPU in the GPU field (<b>Tesla T4</b>, for free users)*
> Seems that PTH models performance better than ONNX for now, you can still try ONNX models and see if it satisfies you


*You can always [click here](https://rentry.co/VoiceChangerGuide#gpu-chart-for-known-working-chunkextra
) to check if these settings are up-to-date*
<br><br>

---

###Always use Colab GPU (**VERY VERY VERY IMPORTANT!**)
You need to use a Colab GPU so the Voice Changer can work faster and better\
Use the menu above and click on **Runtime** » **Change runtime** » **Hardware acceleration** to select a GPU (**T4 is the free one**)

---


# **Credits and Support**
Realtime Voice Changer by [w-okada](https://github.com/w-okada)\
Colab files updated by [rafacasari](https://github.com/Rafacasari)\
Recommended settings by [Raven](https://github.com/ravencutie21)\
Modified again by [Hina](https://github.com/HinaBl)\
Enable FCPE by [TheTrustedComputer](https://github.com/TheTrustedComputer)

Need help? [AI Hub Discord](https://discord.gg/aihub) » ***#help-realtime-vc***


In [1]:
#=================Updated=================
# @title **[1]** Clone repository and install dependencies
# @markdown This first step will download the latest version of Voice Changer and install the dependencies. **It can take some time to complete.(~5min)**

#@markdown ---
# @markdown By using Google Drive, you can avoid re-downloading already downloaded versions.


import os
import time
import subprocess
import threading
import shutil
import base64
import codecs
import torch
import sys

from typing import Literal, TypeAlias

Mode: TypeAlias = Literal["elf", "zip"]
mode:Mode="elf"

# Configs
Run_Cell=0
Use_Drive=False #@param {type:"boolean"}

current_version_hash=None
latest_version_hash=None

# Check GPU
if torch.cuda.is_available():
    print("GPU is available")
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available")
    # sys.exit("No GPU available. Change runtime.")


notebook_env=0
if os.path.exists('/content'):
  notebook_env=1
  print("Welcome to ColabMod")
  from google.colab import drive

elif os.path.exists('/kaggle/working'):
  notebook_env=2
  print("Welcome to Kaggle Mod")
else:
  notebook_env=3
  print("Welcome!")

from IPython.display import clear_output, Javascript

if notebook_env==1 and Use_Drive==True:
  work_dir = "/content/drive/MyDrive/vcclient"
  if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

  if not os.path.exists(work_dir):
    !mkdir -p {work_dir}

  print("Checking latest version...")
  if os.path.exists(f'{work_dir}/latest_hash.txt'):
    current_version_hash = open(f'{work_dir}/latest_hash.txt').read().strip()
  else:
    current_version_hash = None

  !curl -s -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/latest_hash.txt -o latest_hash.txt
  latest_version_hash = open('latest_hash.txt').read().strip()

  print(f"current_version_hash: {current_version_hash}")
  print(f"latest_version_hash : {latest_version_hash}")

  if current_version_hash != latest_version_hash:
    print(f"hash not match -> download latest version")

    latest_hash_path=f'{work_dir}/latest_hash.txt'

    if mode == "elf":
      # !curl -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/vcclient_latest_for_colab -o {work_dir}/vcclient_latest_for_colab
      !curl -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/vcclient_2.0.44-alpha_for_colab -o {work_dir}/vcclient_latest_for_colab

    elif mode == "zip":
      !curl -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/vcclient_latest_for_colab.zip -o {work_dir}/vcclient_latest_for_colab.zip

    !cp latest_hash.txt {latest_hash_path}
    print("Download is done.")
  else:
    print("hash matched. skip download")

else:
  work_dir = "/content"
  print("Downloading the latest vcclient... ")
  !curl -s -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/latest_hash.txt -o latest_hash.txt
  latest_version_hash = open('latest_hash.txt').read().strip()

  if mode == "elf":
    # !curl -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/vcclient_latest_for_colab -o {work_dir}/vcclient_latest_for_colab
    !curl -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/vcclient_2.0.44-alpha_for_colab -o {work_dir}/vcclient_latest_for_colab
  elif mode == "zip":
    !curl -L https://huggingface.co/wok000/vcclient000_colab/resolve/main/vcclient_latest_for_colab.zip -o {work_dir}/vcclient_latest_for_colab.zip

  print("Download is done.")

if current_version_hash != latest_version_hash and mode == "zip":
  print(f"Unzip vcclient to {latest_version_hash} ... ")
  !cd {work_dir} && unzip -q vcclient_latest_for_colab.zip -d {latest_version_hash}
  print(f"Unzip is done.")

if mode == "elf":
  %cd {work_dir}
  !chmod 0700 vcclient_latest_for_colab
elif mode == "zip":
  %cd {work_dir}/{latest_version_hash}/main
  !chmod 0700 main

print("Installing modules... ",end="")
!sudo apt-get install -y libportaudio2 > /dev/null 2>&1
!pip install  pyngrok > /dev/null 2>&1
print("Install is done.")

Run_Cell=1


GPU is available
GPU Name: Tesla T4
Welcome to ColabMod
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1149  100  1149    0     0   3004      0 --:--:-- --:--:-- --:--:--  3007
100 2967M  100 2967M    0     0   154M      0  0:00:19  0:00:19 --:--:--  219M
Download is done.
/content
Installing modules... Install is done.


In [2]:
# @title **[2]** Start server
# @markdown This cell will start the server, the first time that you run it will download the models, so it can take a while (2~4 minutes)


# @markdown ### Options:
ClearConsole = True  # @param {type:"boolean"}
Play_Notification = True  # @param {type:"boolean"}

PORT=8003

LOG_FILE = f"/content/LOG_FILE_{PORT}"

from IPython.display import Audio, display
def play_notification_sound(url):
    display(Audio(url=url, autoplay=True))

from google.colab.output import eval_js


if mode == "elf":
  # !LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/lib/x86_64-linux-gnu ./vcclient_latest_for_colab cui --port {PORT} --no_cui true &

  get_ipython().system_raw(f'LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/lib/x86_64-linux-gnu ./vcclient_latest_for_colab cui --port {PORT} --no_cui true >{LOG_FILE} 2>&1 &')
elif mode == "zip":
  !LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/lib/x86_64-linux-gnu ./main cui --port {PORT} --no_cui true &


import socket
def wait_for_server():
  elapsed_time = 0
  start_time = time.time()


  while True:
      time.sleep(1)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', PORT))
      if result == 0:
          break
      sock.close()
      # 時刻を出力
      current_time = time.time()
      elapsed_time = int(current_time - start_time)
      clear_output(wait=True)
      print(f"Waiting for server... elapsed: {elapsed_time}sec")
      try:
        with open(LOG_FILE, 'r') as f:
            lines = f.readlines()[-5:]
            for line in lines:
                print(line.strip())
      except:
        pass

  if ClearConsole:
      clear_output()
  print("--------- SERVER READY! ---------")
  print(f"Your server is available. elapsed: {elapsed_time}sec")
  proxy = eval_js( "google.colab.kernel.proxyPort(" + str(PORT) + ")" )
  print(f"{proxy}")
  print("---------------------------------")
  if Play_Notification==True:
    play_notification_sound('https://huggingface.co/wok000/voices/resolve/main/vcclient001_vctk229_gpt-sovits_vcclient-ready.wav')
wait_for_server()


--------- SERVER READY! ---------
Your server is available. elapsed: 98sec
https://wdc8fs91tuo-496ff2e9c6d22116-8003-colab.googleusercontent.com/
---------------------------------


In [ ]:
# @title **[FAQ]**

# @markdown #### Q. Delay is too large, why?
# @markdown #### A. In the Colab edition, the audio output buffer is set larger to account for network latency. Depending on the environment, reducing this buffer may reduce delay. However, in some environments, this may cause the sound to become choppy due to network latency. The following diagram shows how to change the buffer size.
# @markdown <img src="https://huggingface.co/wok000/voices/resolve/main/%5Bimg%5Dvcclient001_output_buffer_size_ratio.png?download=true" width="600">
